## 8.1 Handling Highly Correlated Variables

In [ ]:
%config InlineBackend.figure_format = 'svg'
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 70)

### An Initial Linear Model of Online Spend

In [ ]:
import pandas as pd
cust_df = pd.read_csv('http://bit.ly/PMR-ch8pt1')
cust_df.head() # Not shown
cust_df.describe(include='all') # Not shown

In [ ]:
import statsmodels.formula.api as smf
spend_m1 = smf.ols('online_spend ~ age + credit_score + email'
                    '+ distance_to_store + online_visits'
                    '+ online_trans + store_trans + store_spend '
                    '+ sat_service + sat_selection',
                    data=cust_df.loc[cust_df.online_spend > 0,
                                     'age':]).fit()
spend_m1.summary()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_context('paper')

g = sns.PairGrid(cust_df.loc[:, 'age':].fillna(-1), height=1.1)
g.map_upper(plt.scatter, linewidths=1, edgecolor="w", s=5, alpha=0.5)
g.map_diag(plt.hist)
g.map_lower(sns.kdeplot)

In [ ]:
import scipy.stats as ss
import sklearn.preprocessing as pp

def autotransform(x):
  '''Return scaled Box-Cox transform of x'''
  x_bc, lmbd = ss.boxcox(1 + x)
  return pp.scale(x_bc)

In [ ]:
cust_df.head()

In [ ]:
idx_complete = (cust_df.isna().sum(axis=1) == 0)
cust_df_bc = cust_df.loc[(idx_complete) &
                         (cust_df.online_spend > 0), 'age':].copy()
col_idx = cust_df_bc.columns != 'email'
cust_df_bc.iloc[:, col_idx] = \
  cust_df_bc.iloc[:,col_idx].apply(autotransform)

In [ ]:
cust_df_bc.describe(include='all')

In [ ]:
g = sns.PairGrid(cust_df_bc, height=1.1,)
g.map_upper(plt.scatter, linewidths=1, edgecolor="w", s=5, alpha=0.5)
g.map_diag(plt.hist)
g.map_lower(sns.kdeplot) # Not shown

In [ ]:
spend_m2 = smf.ols('online_spend ~ age + credit_score + email'
                    '+ distance_to_store + online_visits'
                    '+ online_trans + store_trans + store_spend '
                    '+ sat_service + sat_selection',
                    data=cust_df_bc).fit()
spend_m2.summary()

In [ ]:
spend_m3 = smf.ols('online_spend ~ online_trans',
                    data=cust_df_bc).fit()
from statsmodels.stats import anova as sms_anova
sms_anova.anova_lm(spend_m2, spend_m3)

### Remediating Collinearity

In [ ]:
from statsmodels.stats.outliers_influence \
  import variance_inflation_factor

variance_inflation_factor(spend_m2.model.exog, 1)

In [ ]:
def print_variance_inflation_factors(model):
  for i, param in enumerate(model.params.index):
    print('VIF: {:.3f}, Parameter: {}'.format(
        variance_inflation_factor(model.model.exog, i), param))

print_variance_inflation_factors(spend_m2)

In [ ]:
spend_m4 = smf.ols('online_spend ~ age + credit_score + email'
                    '+ distance_to_store + online_visits'
                    '+ store_spend + sat_service + sat_selection',
                    data=cust_df_bc).fit()
spend_m4.summary()

In [ ]:
print_variance_inflation_factors(spend_m4)

In [ ]:
from sklearn import decomposition

# Create a combined online variable using PCA
online_pca = (
    decomposition.PCA()
    .fit_transform(
      cust_df_bc[['online_visits','online_trans']]))
cust_df_bc['online'] = online_pca[:,0]

# Create a combined store variable using PCA
store_pca = (
    decomposition.PCA().
    fit_transform(
        cust_df_bc[['store_spend', 'store_trans']]))
cust_df_bc['store'] = store_pca[:,0]

In [ ]:
from sklearn import decomposition

# Create a combined online variable using PCA
online_pca = decomposition.PCA().\
  fit_transform(cust_df_bc[['online_visits','online_trans']])
cust_df_bc['online'] = online_pca[:,0]

# Create a combined store variable using PCA
store_pca = decomposition.PCA().\
  fit_transform(cust_df_bc[['store_spend',
                            'store_trans']])
cust_df_bc['store'] = store_pca[:,0]

In [ ]:
spend_m5 = smf.ols('online_spend ~ age + credit_score + email'
                   '+ distance_to_store + online + store'
                   '+ sat_service + sat_selection',
                    data=cust_df_bc).fit()
spend_m5.summary()

In [ ]:
print_variance_inflation_factors(spend_m5)

## 8.2 Linear Models for Binary Outcomes: Logistic Regression

### 8.2.1 Basics of the Logistic Regression Model

In [ ]:
import numpy as np
np.exp(0) / ( np.exp(0) + 1 )

In [ ]:
from scipy.special import expit
expit(0)

In [ ]:
expit(-np.inf) # infinitely low = likelihood 0

In [ ]:
expit(2) # moderate probability = 88% chance of outcome

In [ ]:
expit(-0.2) # weak likelihood

In [ ]:
np.log(0.88/(1-0.88)) # moderate high likelihood

In [ ]:
from scipy.special import logit
logit(0.88) # equivalent to hand computation

### 8.2.2 Data for Logistic Regression of Season Passes

In [ ]:
pass_df = pd.read_csv('http://bit.ly/PMR-ch8pt2')
pass_df.Pass = pass_df.Pass.astype(
    pd.api.types.CategoricalDtype(categories=['YesPass','NoPass'],
                                  ordered=True))
pass_df.Promo = pass_df.Promo.astype(
    pd.api.types.CategoricalDtype(categories=['NoBundle','Bundle'],
                                  ordered=True))
pass_df.head()

In [ ]:
pass_df.describe()

### 8.2.3 Sales Table Data

In [ ]:
pass_df.groupby(['Pass', 'Promo', 'Channel']).Pass.count().unstack(level=2).T

In [ ]:
channels = ['Mail', 'Park', 'Email']
passes = ['NoPass','YesPass']
promos = ['NoBundle', 'Bundle']

In [ ]:
pass_counts = [278, 449, 359, 242, 49, 223, 284, 639, 485, 83, 27, 38]

In [ ]:
i = 0
pass_array = []
for c in channels:
  for p in passes:
    for b in promos:
      pass_array.append(np.repeat([[c, b, p]], pass_counts[i],
                                  axis=0))
      i += 1

In [ ]:
pass_df = pd.DataFrame(np.concatenate(pass_array),
                       columns=['Channel', 'Promo', 'Pass'])
pass_df.head()

In [ ]:
pass_df.groupby(['Pass', 'Promo', 'Channel']).Pass.count()\
  .unstack(level=2).T

In [ ]:
pass_df.groupby(['Pass', 'Promo']).Pass.count().unstack(level=1)

In [ ]:
pass_df.Pass = pass_df.Pass.astype(
    pd.api.types.CategoricalDtype(categories=['YesPass','NoPass'],
                                  ordered=True))
pass_df.Promo = pass_df.Promo.astype(
    pd.api.types.CategoricalDtype(categories=['NoBundle','Bundle'],
                                  ordered=True))

In [ ]:
pass_df.groupby(['Pass', 'Promo']).Pass.count().unstack(level=1)

### 8.2.4 Fitting a Logistic Regression Model

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

pass_m1 = smf.glm('Pass ~ Promo', data=pass_df,
                  family=sm.families.Binomial()).fit()
pass_m1.summary()

In [ ]:
# ratio of outcome % to alternative %
expit(0.3888) / (1-expit(0.3888))

In [ ]:
np.exp(0.3888) # identical

In [ ]:
print('Odds of pass:no pass, bundle: {:.3f} : 1'
  .format(np.exp(0.3888 - 0.1922)))
print('Odds of pass:no pass, without bundle: {:.3f} : 1'
  .format(np.exp(-0.1922)))

In [ ]:
np.exp(0.3888 - 0.1922)/(1 + np.exp(0.3888 - 0.1922))

In [ ]:
prob_pass_with_bundle = (np.exp(0.3888 - 0.1922)/
                         (1 + np.exp(0.3888 - 0.1922)))
print('Probability of pass, bundle: {:.3f}'
  .format(prob_pass_with_bundle))
prob_pass_without_bundle = np.exp(-0.1922)/(1 + np.exp(-0.1922))
print('Probability of pass, no bundle: {:.3f}'
  .format(prob_pass_without_bundle))
print('Odds ratio: {:.3f}'
.format((prob_pass_with_bundle/(1-prob_pass_with_bundle))
        /(prob_pass_without_bundle/(1-prob_pass_without_bundle))))
print('Odds ratio: {:.3f}'.format(np.exp(0.3888)))

In [ ]:
np.exp(pass_m1.params)

In [ ]:
np.exp(pass_m1.conf_int())

### 8.2.5 Reconsidering the model

In [ ]:
pass_df.groupby(['Pass']).Channel.value_counts().unstack()

In [ ]:
pass_df.groupby(['Pass']).Channel.value_counts().unstack()\
  .plot(kind='barh', stacked=True, figsize=(10,6))

In [ ]:
f = plt.figure(figsize=(15,8))
axs = f.subplots(1,3)
for ax, (c, channel_group) in zip(axs, pass_df.groupby("Channel")):
  ax = channel_group\
   .groupby("Promo")\
   .Pass.value_counts(normalize=True).unstack()\
   .plot(kind='bar', ax=ax, stacked=True)
  ax.set_title(c)

In [ ]:
channels = ['Mail', 'Park', 'Email']
plt.figure(figsize=(15,8))
for i,c in enumerate(channels):
  ax = plt.subplot(1, 3, i+1)
  pass_df.loc[pass_df.Channel == c]\
    .groupby('Promo').Pass.value_counts(normalize=True).unstack()\
    .plot(kind='bar', ax=ax, stacked=True)
  plt.title(c)
  plt.ylim((0,1.3))

In [ ]:
pass_m2 = smf.glm('Pass ~ Promo + Channel',
                  data=pass_df,
                  family=sm.families.Binomial()).fit()
pass_m2.summary()

In [ ]:
np.exp(pass_m2.params)

In [ ]:
np.exp(pass_m2.conf_int())

In [ ]:
pass_m3 = smf.glm('Pass ~ Promo + Channel + Promo:Channel',
                  data=pass_df,
                  family=sm.families.Binomial()).fit()
pass_m3.summary()

In [ ]:
np.exp(pass_m3.conf_int())

## 8.3 An introduction to Hierarchical Models

### Ratings-Based Conjoint Analysis for the Amusement Park

In [ ]:
conjoint_df = pd.read_csv('http://bit.ly/PMR-ch8pt3')
conjoint_df.speed = conjoint_df.speed.astype('category')
conjoint_df.height = conjoint_df.height.astype('category')
conjoint_df.head() # Not shown
conjoint_df.describe(include='all')

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(89745)
response_id = range(200) # respondent ids
n_questions = 16 # number of conjoint ratings per respondent
speed_options = ['40', '50', '60', '70']
speed = np.random.choice(speed_options,
                         size=n_questions,
                         replace=True)
height_options = ['200', '300', '400']
height = np.random.choice(height_options,
                         size=n_questions,
                         replace=True)
const_options = ['Steel', 'Wood']
const= np.random.choice(const_options,
                         size=n_questions,
                         replace=True)
theme_options = ['Dragon', 'Eagle']
theme = np.random.choice(theme_options,
                         size=n_questions,
                         replace=True)

In [ ]:
profiles_df = pd.DataFrame([speed, height, const, theme],
                           index=['speed', 'height',
                                  'const', 'theme']).T
profiles_df

In [ ]:
profile_dummies = pd.get_dummies(profiles_df)
profile_dummies.drop(
    ['speed_40', 'height_200', 'const_Steel', 'theme_Dragon'],
    axis=1, inplace=True)
profiles_model = pd.concat(
    [pd.Series(np.ones(16, dtype=int), name='Intercept'),
     profile_dummies],
    axis=1)
profiles_model

In [ ]:
weights = np.random.multivariate_normal(
    mean=[-3, 0.5, 1, 3, 2, 1, -0.2, -0.5],
    cov=np.diag([0.2, 0.1, 0.1, 0.1, 0.2, 0.3, 1, 1]),
    size=len(response_id)
)

In [ ]:
conjoint_df = pd.DataFrame()
for i in response_id:
  utility = (
      (profiles_model * weights[i]).sum(axis=1)
      + np.random.normal(size=16))
  ratings = pd.cut(utility, 10, labels=range(1,11))
  conjoint_resp = profiles_df.copy()
  conjoint_resp['rating'] = pd.to_numeric(ratings)
  conjoint_resp['resp_id'] = i
  conjoint_df = conjoint_df.append(conjoint_resp,
                                   ignore_index=True)
conjoint_df.head()

In [ ]:
conjoint_df.describe(include='all')

### 8.3.4 An Initial Linear Model

In [ ]:
conjoint_df.groupby('height').rating.mean()

In [ ]:
import statsmodels.formula.api as smf
ride_lm = smf.ols('rating ~ speed + height + const + theme',
                    data=conjoint_df).fit()
ride_lm.summary()

### 8.3.5 Hierarchical Linear Model with statsmodels

In [ ]:
ride_hlm_1 = smf.mixedlm('rating ~ speed + height + const + theme',
                         data=conjoint_df,
                         groups=conjoint_df['resp_id'],
                         re_formula='~ 1')
ride_hlm_1_f = ride_hlm_1.fit()
ride_hlm_1_f.summary()

In [ ]:
ride_hlm_1_f.fe_params

In [ ]:
re_params = pd.DataFrame(ride_hlm_1_f.random_effects).T
re_params.head()

In [ ]:
ride_hlm_1_f_coef = \
  ride_hlm_1_f.fe_params.to_frame().T\
    .iloc[np.zeros(len(re_params))]
ride_hlm_1_f_coef.index = range(len(re_params))
ride_hlm_1_f_coef.Intercept += re_params.Group

ride_hlm_1_f_coef.head()

### 8.3.6 The Complete Hierarchical Linear Model

In [ ]:
np.random.seed(89745)
ride_hlm_2 = smf.mixedlm('rating ~ speed + height + const + theme',
                         data=conjoint_df,
                         groups=conjoint_df['resp_id'],
                         re_formula='~ speed + height + const + theme')
ride_hlm_2_f = ride_hlm_2.fit(maxiter=1000, method='nm')

In [ ]:
ride_hlm_2_f.fe_params

In [ ]:
ride_hlm_2_f_re_df = pd.DataFrame(ride_hlm_2_f.random_effects).T
ride_hlm_2_f_re_df.rename({'Group': 'Intercept'},
                          axis=1, inplace=True)
ride_hlm_2_f_re_df.head()

### 8.3.7 Interpreting random effects

In [ ]:
hlm_2_f_coef = ride_hlm_2_f_re_df + ride_hlm_2_f.fe_params
hlm_2_f_coef.head()

In [ ]:
sns.heatmap(hlm_2_f_coef.iloc[:,1:].corr(), vmax=0.3)


In [ ]:
sns.heatmap(ride_hlm_2_f_re_df.iloc[:,1:].corr(), vmax=0.3)

In [ ]:
cg = sns.clustermap(ride_hlm_2_f_re_df.iloc[:,1:].corr(), vmax=0.5,
                    vmin=-0.5,cmap=plt.cm.bwr, center=0)
plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels(), rotation=45)